# Coursera Project On Toronto Neighborhood Segmentation

### Will showcase web-scraping and geo-spatial clustering

In [5]:
#importing libraries that will be used
import pandas as pd #data analysis
#ensure that we can print out an entire dataset if desired
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import requests #handle web requests
from bs4 import BeautifulSoup #parse the resultant html

In [6]:
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #link containing our data to be analyzed
table_class = "wikitable sortable jquery-tablesorter" #define type of table expected to be collected from target link
response = requests.get(wikiurl) #retrieve html data from target link
print(response.status_code) #HTML status code, 200 is expected as that indicates succescful retrieval

200


In [7]:
#convert the html data into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
#specify table object from html response
toronto_table=soup.find('table', {'class':"wikitable"})
toronto_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

From the output of the previous cell, we can see that the table is being located with the desired Toronto neighborhood data.

In [9]:
#convert BeautifulSoup object to Pandas DataFrame 
df_toronto = pd.read_html(str(toronto_table)) #using Pandas built in function for reading html
df_toronto = pd.DataFrame(df_toronto[0]) #coerce list into DataFrame
print(df_toronto.head()) #verify

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


The output of the cell above demonstrates that the table contains the features that we are looking for (Postal Code, Borough, Neighborhood), however it contains several instances on "Not assigned" data. For this, we will need to clean the data.

Since our goal is to segment Toronto neighborhoods, entry points with no neighborhoods are not useful to us; however, we can substitue the borough in cases where a neighborhood is not assigned. If no borough is available either, it will be necessary to drop the entry.

In [10]:
#Begin cleaning data
NoBoroughIndexes = df_toronto[df_toronto['Borough'] == 'Not assigned'].index #establish indexes where no borough is assigned
df_toronto_bor = df_toronto.drop(NoBoroughIndexes) #drop the indexes obtained
df_toronto_bor.head() #verify succesful drop

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In the output above, we can see that we have succesfully removed rows with no borough assigned, however,we should check to see if there is any data with a borough but no neighborhood.

In [11]:
print(df_toronto_bor[df_toronto_bor['Neighbourhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


We can see above that parsing the neighborhood column for 'Not assigned' entries returns nothing. However, we can double check to make sure there are no rows with unassigned neighborhoods under different entries.

In [12]:
df_neighborhoods = pd.DataFrame(df_toronto_bor['Neighbourhood'].value_counts())
df_neighborhoods

,Neighbourhood
Downsview,4
Don Mills,2
"New Toronto, Mimico South, Humber Bay Shores",1
Canada Post Gateway Processing Centre,1
Victoria Village,1
Stn A PO Boxes,1
Roselawn,1
Christie,1
"Guildwood, Morningside, West Hill",1
Lawrence Park,1


We can see when looking at the counts for how many times a neighborhood shows up, there are no entries without assigned neighborhoods.
We can also see that many postal areas contain multiple nighborhoods, where neighboroods are separated by commas in the above table.

In [13]:
shape = df_toronto_bor.shape
row = shape[0]
col = shape[1]
fmt_str = "There are %i rows and %i columns in our cleaned data set" % (row, col)
print(fmt_str)

There are 103 rows and 3 columns in our cleaned data set


# Part Two:
## In this section (which will be saved as a new and separate notebook, as per the assignment's instructions), we will find the latitude and longitude data of the previously identified boroughs and neighborhoods

In [12]:
!pip install geocoder

In [15]:
#To do this, we are going to use geocoder
import geocoder

#we will start with an empty lat/long column
df_toronto_bor['Lat/Long'] = None
df_toronto_bor.reset_index(inplace=True)
df_toronto_bor.drop(['level_0', 'index'], axis=1, inplace=True)

In [16]:
df_toronto_bor.head()

,Postal Code,Borough,Neighbourhood,Lat/Long
0,M3A,North York,Parkwoods,None
1,M4A,North York,Victoria Village,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None


In [17]:
#We can then construct a loop to populate each postal code with a latitude and longitude value

#for code, data in (df_toronto_bor['Postal Code'].items()):
#    while(df_toronto_bor.loc[code, 'Lat/Long'] is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(data))
#        df_toronto_bor.loc[0, 'Lat/Long'] = g.latlng


In [34]:
print(lat_lng_coords)
print(g)

None
<[REQUEST_DENIED] Google - Geocode [empty]>


Here we can see that the Geocode library is producing no results, as indicated by the instructions for this project. For this, the loop that would be used to generate the lat/long for each postal code has been commented out.

We will proceed using the CSV file of lat/long values instead

In [37]:
#We will have to import a library, io, to decode the CSV at the link provided
import io
csv_url = 'https://cocl.us/Geospatial_data' #Link to CSV
csv_response = requests.get(csv_url).content #Retrieve content at the link
ll_df = pd.read_csv(io.StringIO(csv_response.decode('utf-8'))) # Decode the content string and coerce it to df
ll_df.head() #Preview df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
#drop lat/long column made before failed loop 
df_toronto_bor.drop(['Lat/Long'], axis=1, inplace=True)
#merge original table with lat/long table
neighborhood_ll = pd.merge(df_toronto_bor, ll_df, on=['Postal Code', 'Postal Code'])
neighborhood_ll.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Here, we can see that the provided CSV has been uploaded and merged into the main dataframe, so now we have the postal code, corresponding borough, neighborhood, and latitude/longitude values for each.